In [1]:
#Import Libraries
import os
import pathlib
import earthpy.earthexplorer as etee
import earthpy.appeears as etapp
import earthpy as et
import geopandas as gpd
import requests
import zipfile
import io
import matplotlib.pyplot as plt

In [2]:
# Create directory et.io.HOME
data_dir = os.path.join(et.io.HOME, 'final','data')
print(data_dir)


/home/jovyan/final/data


In [3]:
# Code adapted from: https://medium.com/@loldja/reading-shapefile-zips-from-a-url-in-python-3-93ea8d727856
# Create directory
grassland_url = 'https://data.fs.usda.gov/geodata/edw/edw_resources/shp/S_USA.NationalGrassland.zip'
print('Downloading shapefile...')
# Request data from url
grassland_request = requests.get(grassland_url)
grassland_zip = zipfile.ZipFile(io.BytesIO(grassland_request.content))
print("Done")
# Extract files from Zip to 
grassland_zip.extractall(path=os.path.join(data_dir, 'national-grassland'))
# filenames = [y for y in sorted(grassland_zip.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
# print(filenames)



Done


In [4]:
# Import shapefile
grassland_gdf = gpd.read_file(os.path.join(data_dir, 'national-grassland', 'S_USA.NationalGrassland.shp'))
select_grassland_gdf = (
    grassland_gdf
    .set_index('GRASSLANDN')
    .loc[['Comanche National Grassland', 'Pawnee National Grassland']]
)
select_grassland_gdf

,NATIONALGR,GIS_ACRES,SHAPE_AREA,SHAPE_LEN,geometry
GRASSLANDN,,,,,
Comanche National Grassland,295522010328,444413.904,0.183064,26.658022,"MULTIPOLYGON (((-104.02263 37.69224, -104.0225..."
Pawnee National Grassland,295523010328,208424.885,0.089972,15.341594,"MULTIPOLYGON (((-104.58106 40.82664, -104.5810..."


In [ ]:
# Get Polaris Data
POLARIS dataset

# https://github.com/njadid/Polaris_Processor

In [5]:
select_grassland_gdf

select_grassland_gdf = select_grassland_gdf.to_crs(4326)

In [9]:
# https://github.com/njadid/Polaris_Processor?

def download_polaris(name, geometry, start, end):
    """
    Download NAIP raster for a given geometry, start date, and end date

    Downloads data from the National Agricultural Imagery Program (NAIP)
    given a spatial and temporal extent.
    <citation>

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """
    print(f'Site Name: {name}')
    # Create bounding box
    bbox = etee.BBox(*geometry.bounds)
    # Create downloader
    naip_downloader = etee.EarthExplorerDownloader(
        dataset="Polaris", 
        label=name.lower().replace(" ", "-"),
        bbox=bbox,
        start=start,
        end=end,
        store_credential=True)
    # Request and download data
    naip_downloader.submit_download_request()
    naip_downloader.download(override=False)
    return naip_downloader

# Run to test
for site_name, details in select_grassland_gdf.iterrows():
  downloader = download_polaris(
     site_name, details.geometry, '2022-01-01', '2022-12-31')

Site Name: Caddo National Grassland
Login Successful.
Searching datasets...


TypeError: object of type 'NoneType' has no len()

In [13]:
# https://lpdaac.usgs.gov/products/srtmgl1v003/
# https://appeears.earthdatacloud.nasa.gov/api/#temporary-s3-credentials
# https://appeears.earthdatacloud.nasa.gov/products
# https://lpdaac.usgs.gov/resources/e-learning/getting-started-with-the-a%CF%81%CF%81eears-api-submitting-and-downloading-an-area-request/

# Download Elevation Data Using Appeears
elevation_downloader = etapp.AppeearsDownloader(
    download_key="retry3",
    ea_dir=data_dir,
    product="SRTMGL1_NC.003",
    layer="SRTMGL1_DEM",
    start_date="02-11",
    end_date="02-21",
    recurring=True,
    year_range=[2000, 2000],
    polygon=select_grassland_gdf,
)

# Download files if the download directory does not exist
if not os.path.exists(elevation_downloader.data_dir):
    elevation_downloader.download_files()

### Data Citations
NASA JPL (2013). NASA Shuttle Radar Topography Mission Global 1 arc second [Data set]. NASA EOSDIS Land Processes Distributed Active Archive Center. Accessed 2023-12-05 from https://doi.org/10.5067/MEaSUREs/SRTM/SRTMGL1.003
